Looking for best approach to feature imputation

Constants:
* Not using parquet fields
* Not using PCA
* Non-augmented data
* Feature selection, calculation
* Using single XGBoost model for predictions from imputed labelled data
* StandardScaler

Varying:
* Feature imputation model: mean (control), median (control), KNN, MICE, RandomForest, autoencoder

In [1]:
import numpy as np
import pandas as pd

import os

import polars as pl
from glob import glob
from tqdm.auto import tqdm

import tensorflow as tf
from tensorflow import keras
import random

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn.svm import SVR

import lightgbm as lgb

from sklearn.model_selection import KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [2]:
train_data=pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test_data = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

In [3]:
INPUT_DIR = "/kaggle/input/child-mind-institute-problematic-internet-use/"

# Import aggregate fields from parquet files
# Modified code from rsakata: https://www.kaggle.com/code/rsakata/cmi-piu-16th-place-solution

files_train = glob(INPUT_DIR + "series_train.parquet/*")
#if IS_SUBMIT:
#    files += glob(INPUT_DIR + "series_test.parquet/*")

list_df_train = []
for file in tqdm(files_train):
    df_series = (
        pl.read_parquet(file)
        .with_columns(
            (
                (pl.col("relative_date_PCIAT") - pl.col("relative_date_PCIAT").min())*24
                + (pl.col("time_of_day") // int(1e9)) / 3600
            ).floor().cast(int).alias("total_hours")
        )
        .filter(pl.col("non-wear_flag") != 1)
        .filter(pl.col("step").count().over("total_hours") == 12 * 60)
        .group_by("total_hours").agg(
            pl.col("enmo").std().alias("enmo_std"),
            pl.col("anglez").std().alias("anglez_std"),
            pl.col("light").std().alias("light_std")
        )
        .with_columns(
            (pl.col("total_hours") % 24).alias("hour"),
            pl.lit(file.split("/")[-1][3:]).alias("id")
        )
    )
    list_df_train.append(df_series.to_pandas())

df_series = pd.concat(list_df_train)
df_series["enmo_std"] = np.log(df_series["enmo_std"] + 0.01)
df_series["anglez_std"] = np.log(df_series["anglez_std"] + 1)
df_series["light_std"] = np.log(df_series["light_std"] + 0.01)

df_agg_train = df_series.groupby("id")[["enmo_std", "anglez_std", "light_std"]].agg(["mean", "std"]).reset_index()
df_agg_train.columns = [cols[0] + "_" + cols[1] if cols[1] != "" else cols[0] for cols in df_agg_train.columns]
df_agg_train

  0%|          | 0/996 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,enmo_std_mean,enmo_std_std,anglez_std_mean,anglez_std_std,light_std_mean,light_std_std
0,00115b9f,-4.000377,NaN,1.989905,NaN,0.051475,NaN
1,001f3379,-3.514671,0.652348,3.236993,0.678888,0.774591,2.945807
2,00f332d1,-3.071176,0.927238,3.249122,0.463244,1.138379,2.939823
3,01085eb3,-2.902040,0.791255,3.389762,0.315061,1.054698,2.185839
4,012cadd8,-2.806918,1.171675,3.337322,0.388409,0.823770,3.350365
...,...,...,...,...,...,...,...
964,fe9c71d8,-3.116904,0.961804,3.037607,0.943554,-0.394200,2.742634
965,fecc07d6,-3.969482,0.981531,1.332831,1.428363,-0.438018,1.795653
966,ff18b749,-2.820076,0.937540,3.258458,0.417267,1.236652,3.341580
967,ffcd4dbd,-3.271800,0.827489,3.183395,0.629553,0.521227,2.665325


In [4]:
train_data2 = train_data.merge(df_agg_train, how="left", on="id")
train_data2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,enmo_std_mean,enmo_std_std,anglez_std_mean,anglez_std_std,light_std_mean,light_std_std
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,NaN,Fall,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,64.0,Summer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,54.0,Summer,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.0,Winter,0.0,1.0,-4.000377,NaN,1.989905,NaN,0.051475,NaN
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
files_test = glob(INPUT_DIR + "series_test.parquet/*")


list_df_test = []
for file in tqdm(files_test):
    df_series = (
        pl.read_parquet(file)
        .with_columns(
            (
                (pl.col("relative_date_PCIAT") - pl.col("relative_date_PCIAT").min())*24
                + (pl.col("time_of_day") // int(1e9)) / 3600
            ).floor().cast(int).alias("total_hours")
        )
        .filter(pl.col("non-wear_flag") != 1)
        .filter(pl.col("step").count().over("total_hours") == 12 * 60)
        .group_by("total_hours").agg(
            pl.col("enmo").std().alias("enmo_std"),
            pl.col("anglez").std().alias("anglez_std"),
            pl.col("light").std().alias("light_std")
        )
        .with_columns(
            (pl.col("total_hours") % 24).alias("hour"),
            pl.lit(file.split("/")[-1][3:]).alias("id")
        )
    )
    list_df_test.append(df_series.to_pandas())

df_series = pd.concat(list_df_test)
df_series["enmo_std"] = np.log(df_series["enmo_std"] + 0.01)
df_series["anglez_std"] = np.log(df_series["anglez_std"] + 1)
df_series["light_std"] = np.log(df_series["light_std"] + 0.01)

df_agg_test = df_series.groupby("id")[["enmo_std", "anglez_std", "light_std"]].agg(["mean", "std"]).reset_index()
df_agg_test.columns = [cols[0] + "_" + cols[1] if cols[1] != "" else cols[0] for cols in df_agg_test.columns]
df_agg_test

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,enmo_std_mean,enmo_std_std,anglez_std_mean,anglez_std_std,light_std_mean,light_std_std
0,00115b9f,-4.000377,NaN,1.989905,NaN,0.051475,NaN
1,001f3379,-3.514671,0.652348,3.236993,0.678888,0.774591,2.945807


In [6]:
test_data2 = test_data.merge(df_agg_test, how="left", on="id")
test_data2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,enmo_std_mean,enmo_std_std,anglez_std_mean,anglez_std_std,light_std_mean,light_std_std
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,NaN,NaN,Fall,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,46.0,64.0,Summer,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,38.0,54.0,Summer,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,31.0,45.0,Winter,0.0,-4.000377,NaN,1.989905,NaN,0.051475,NaN
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
X_train = train_data2[['Basic_Demos-Age',
                      'Basic_Demos-Sex',
                      'CGAS-CGAS_Score',
                      'Physical-BMI',
                      'BIA-BIA_BMI',
                      'Physical-Waist_Circumference',
                      'Physical-Diastolic_BP',
                      'Physical-HeartRate',
                      'Physical-Systolic_BP',
                      'Fitness_Endurance-Max_Stage',
                      'Fitness_Endurance-Time_Mins',
                      'Fitness_Endurance-Time_Sec',
                      'FGC-FGC_CU_Zone',
                      'FGC-FGC_GSND_Zone',
                      'FGC-FGC_GSD_Zone',
                      'FGC-FGC_PU_Zone',
                      'FGC-FGC_SRL_Zone',
                      'FGC-FGC_SRR_Zone',
                      'FGC-FGC_TL_Zone',
                      'BIA-BIA_Activity_Level_num',
                      'BIA-BIA_BMC',
                      'BIA-BIA_BMR',
                      'BIA-BIA_DEE',
                      'BIA-BIA_ECW',
                      'BIA-BIA_FFM',
                      'BIA-BIA_FFMI',
                      'BIA-BIA_FMI',
                      'BIA-BIA_Fat',
                      'BIA-BIA_ICW',
                      'BIA-BIA_LDM',
                      'BIA-BIA_LST',
                      'BIA-BIA_SMM',
                      'BIA-BIA_TBW',
                      'PAQ_A-PAQ_A_Total',
                      'PAQ_C-PAQ_C_Total',
                      'SDS-SDS_Total_T',
                      'PreInt_EduHx-computerinternet_hoursday'
                      # ,
                      #'enmo_std_mean',
                      #'enmo_std_std',
                      #'anglez_std_mean',
                      #'anglez_std_std',
                      #'light_std_mean',
                      #'light_std_std',
                      ]]

y_train = train_data2['PCIAT-PCIAT_Total']

X_test = test_data2[['Basic_Demos-Age',
                      'Basic_Demos-Sex',
                      'CGAS-CGAS_Score',
                      'Physical-BMI',
                      'BIA-BIA_BMI',
                      'Physical-Waist_Circumference',
                      'Physical-Diastolic_BP',
                      'Physical-HeartRate',
                      'Physical-Systolic_BP',
                      'Fitness_Endurance-Max_Stage',
                      'Fitness_Endurance-Time_Mins',
                      'Fitness_Endurance-Time_Sec',
                      'FGC-FGC_CU_Zone',
                      'FGC-FGC_GSND_Zone',
                      'FGC-FGC_GSD_Zone',
                      'FGC-FGC_PU_Zone',
                      'FGC-FGC_SRL_Zone',
                      'FGC-FGC_SRR_Zone',
                      'FGC-FGC_TL_Zone',
                      'BIA-BIA_Activity_Level_num',
                      'BIA-BIA_BMC',
                      'BIA-BIA_BMR',
                      'BIA-BIA_DEE',
                      'BIA-BIA_ECW',
                      'BIA-BIA_FFM',
                      'BIA-BIA_FFMI',
                      'BIA-BIA_FMI',
                      'BIA-BIA_Fat',
                      'BIA-BIA_ICW',
                      'BIA-BIA_LDM',
                      'BIA-BIA_LST',
                      'BIA-BIA_SMM',
                      'BIA-BIA_TBW',
                      'PAQ_A-PAQ_A_Total',
                      'PAQ_C-PAQ_C_Total',
                      'SDS-SDS_Total_T',
                      'PreInt_EduHx-computerinternet_hoursday'
                      #,
                      #'enmo_std_mean',
                      #'enmo_std_std',
                      #'anglez_std_mean',
                      #'anglez_std_std',
                      #'light_std_mean',
                      #'light_std_std',
                   ]]

In [27]:
# Add calculated fields
X_train['Physical-BMI_Calc'] = X_train.apply(lambda row: row['Physical-BMI'] if row['Physical-BMI']==row['Physical-BMI'] else row['BIA-BIA_BMI'],axis=1)
X_train['Fitness_Endurance-Time_Sec_Calc'] = X_train.apply(lambda row: row['Fitness_Endurance-Time_Sec'] + (row['Fitness_Endurance-Time_Mins']*60), axis=1)
X_train['PAQ_Total'] = X_train.apply(lambda row: row['PAQ_A-PAQ_A_Total'] if row['PAQ_A-PAQ_A_Total']==row['PAQ_A-PAQ_A_Total'] else row['PAQ_C-PAQ_C_Total'],axis=1)


# Drop fields no longer needed
X_train = X_train.drop(columns=['PAQ_A-PAQ_A_Total','PAQ_C-PAQ_C_Total',
                     'Physical-BMI','BIA-BIA_BMI',
                     'Fitness_Endurance-Time_Mins','Fitness_Endurance-Time_Sec'])

# Remove outliers - may give warnings due to NaN value comparison
X_train.loc[X_train['CGAS-CGAS_Score']>=100.0,'CGAS-CGAS_Score'] = np.nan
X_train.loc[X_train['Physical-Systolic_BP']>=180.0,'Physical-Systolic_BP'] = np.nan
X_train.loc[X_train['Physical-Diastolic_BP']>=120.0,'Physical-Diastolic_BP'] = np.nan
X_train.loc[X_train['BIA-BIA_DEE']>=6000.0,'BIA-BIA_DEE'] = np.nan
X_train.loc[(X_train['BIA-BIA_BMC']<=0.0) | (X_train['BIA-BIA_BMC']>=16.0),'BIA-BIA_BMC'] = np.nan
X_train.loc[(X_train['BIA-BIA_BMR']<=0.0) | (X_train['BIA-BIA_BMR']>=2400.0),'BIA-BIA_BMR'] = np.nan
X_train.loc[(X_train['BIA-BIA_ECW']<=0.0) | (X_train['BIA-BIA_ECW']>=60.0),'BIA-BIA_ECW'] = np.nan
X_train.loc[(X_train['BIA-BIA_FFM']<=0.0) | (X_train['BIA-BIA_FFM']>=200.0),'BIA-BIA_FFM'] = np.nan
X_train.loc[(X_train['BIA-BIA_FFMI']<=0.0) | (X_train['BIA-BIA_FFMI']>=25.0),'BIA-BIA_FFMI'] = np.nan
X_train.loc[(X_train['BIA-BIA_FMI']<=0.0) | (X_train['BIA-BIA_FMI']>=25.0),'BIA-BIA_FMI'] = np.nan
X_train.loc[(X_train['BIA-BIA_Fat']<=8.0) | (X_train['BIA-BIA_Fat']>=60.0),'BIA-BIA_Fat'] = np.nan
X_train.loc[(X_train['BIA-BIA_ICW']<=0.0) | (X_train['BIA-BIA_ICW']>=80.0),'BIA-BIA_ICW'] = np.nan
X_train.loc[(X_train['BIA-BIA_LDM']<=0.0) | (X_train['BIA-BIA_LDM']>=60.0),'BIA-BIA_LDM'] = np.nan
X_train.loc[(X_train['BIA-BIA_LST']<=0.0) | (X_train['BIA-BIA_LST']>=150.0),'BIA-BIA_LST'] = np.nan
X_train.loc[(X_train['BIA-BIA_SMM']<=0.0) | (X_train['BIA-BIA_SMM']>=100.0),'BIA-BIA_SMM'] = np.nan
X_train.loc[(X_train['BIA-BIA_TBW']<=0.0) | (X_train['BIA-BIA_TBW']>=150.0),'BIA-BIA_TBW'] = np.nan

<ipython-input-27-7507d9027830>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Physical-BMI_Calc'] = X_train.apply(lambda row: row['Physical-BMI'] if row['Physical-BMI']==row['Physical-BMI'] else row['BIA-BIA_BMI'],axis=1)
<ipython-input-27-7507d9027830>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Fitness_Endurance-Time_Sec_Calc'] = X_train.apply(lambda row: row['Fitness_Endurance-Time_Sec'] + (row['Fitness_Endurance-Time_Mins']*60), axis=1)
<ipython-input-27-7507d9027830>:4: Settin

In [28]:
# Repeat the above for X_test

# Add calculated fields
X_test['Physical-BMI_Calc'] = X_test.apply(lambda row: row['Physical-BMI'] if row['Physical-BMI']==row['Physical-BMI'] else row['BIA-BIA_BMI'],axis=1)
X_test['Fitness_Endurance-Time_Sec_Calc'] = X_test.apply(lambda row: row['Fitness_Endurance-Time_Sec'] + (row['Fitness_Endurance-Time_Mins']*60), axis=1)
X_test['PAQ_Total'] = X_test.apply(lambda row: row['PAQ_A-PAQ_A_Total'] if row['PAQ_A-PAQ_A_Total']==row['PAQ_A-PAQ_A_Total'] else row['PAQ_C-PAQ_C_Total'],axis=1)

# Drop fields no longer needed
X_test = X_test.drop(columns=['PAQ_A-PAQ_A_Total','PAQ_C-PAQ_C_Total',
                     'Physical-BMI','BIA-BIA_BMI',
                     'Fitness_Endurance-Time_Mins','Fitness_Endurance-Time_Sec'])

# Remove outliers
X_test.loc[X_test['CGAS-CGAS_Score']>=100.0,'CGAS-CGAS_Score'] = np.nan
X_test.loc[X_test['Physical-Systolic_BP']>=180.0,'Physical-Systolic_BP'] = np.nan
X_test.loc[X_test['Physical-Diastolic_BP']>=120.0,'Physical-Diastolic_BP'] = np.nan
X_test.loc[X_test['BIA-BIA_DEE']>=6000.0,'BIA-BIA_DEE'] = np.nan
X_test.loc[(X_test['BIA-BIA_BMC']<=0.0) | (X_test['BIA-BIA_BMC']>=16.0),'BIA-BIA_BMC'] = np.nan
X_test.loc[(X_test['BIA-BIA_BMR']<=0.0) | (X_test['BIA-BIA_BMR']>=2400.0),'BIA-BIA_BMR'] = np.nan
X_test.loc[(X_test['BIA-BIA_ECW']<=0.0) | (X_test['BIA-BIA_ECW']>=60.0),'BIA-BIA_ECW'] = np.nan
X_test.loc[(X_test['BIA-BIA_FFM']<=0.0) | (X_test['BIA-BIA_FFM']>=200.0),'BIA-BIA_FFM'] = np.nan
X_test.loc[(X_test['BIA-BIA_FFMI']<=0.0) | (X_test['BIA-BIA_FFMI']>=25.0),'BIA-BIA_FFMI'] = np.nan
X_test.loc[(X_test['BIA-BIA_FMI']<=0.0) | (X_test['BIA-BIA_FMI']>=25.0),'BIA-BIA_FMI'] = np.nan
X_test.loc[(X_test['BIA-BIA_Fat']<=8.0) | (X_test['BIA-BIA_Fat']>=60.0),'BIA-BIA_Fat'] = np.nan
X_test.loc[(X_test['BIA-BIA_ICW']<=0.0) | (X_test['BIA-BIA_ICW']>=80.0),'BIA-BIA_ICW'] = np.nan
X_test.loc[(X_test['BIA-BIA_LDM']<=0.0) | (X_test['BIA-BIA_LDM']>=60.0),'BIA-BIA_LDM'] = np.nan
X_test.loc[(X_test['BIA-BIA_LST']<=0.0) | (X_test['BIA-BIA_LST']>=150.0),'BIA-BIA_LST'] = np.nan
X_test.loc[(X_test['BIA-BIA_SMM']<=0.0) | (X_test['BIA-BIA_SMM']>=100.0),'BIA-BIA_SMM'] = np.nan
X_test.loc[(X_test['BIA-BIA_TBW']<=0.0) | (X_test['BIA-BIA_TBW']>=150.0),'BIA-BIA_TBW'] = np.nan

<ipython-input-28-ee14b5292b05>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Physical-BMI_Calc'] = X_test.apply(lambda row: row['Physical-BMI'] if row['Physical-BMI']==row['Physical-BMI'] else row['BIA-BIA_BMI'],axis=1)
<ipython-input-28-ee14b5292b05>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Fitness_Endurance-Time_Sec_Calc'] = X_test.apply(lambda row: row['Fitness_Endurance-Time_Sec'] + (row['Fitness_Endurance-Time_Mins']*60), axis=1)
<ipython-input-28-ee14b5292b05>:6: SettingWit

# Feature imputation models

In [29]:
X_train.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,2420.000000,898.000000,2935.000000,2967.000000,2938.000000,743.000000,2282.000000,1062.000000,...,1978.000000,1982.000000,1963.000000,1974.000000,1981.000000,2606.000000,3301.000000,3042.000000,739.000000,2195.000000
mean,10.433586,0.372727,65.069008,27.278508,69.242249,81.597236,116.591899,4.989233,0.476337,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,57.763622,1.060588,19.334696,469.910690,2.500797
std,3.574648,0.483591,11.787310,5.567287,12.631379,13.665196,16.252453,2.014072,0.499549,0.612585,...,9.746636,7.091770,23.638407,13.855726,19.570096,13.196091,1.094875,5.104280,188.716073,0.816265
min,5.000000,0.000000,25.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,1.000000,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,0.000000,0.000000,5.000000,0.580000
25%,8.000000,0.000000,59.000000,23.000000,61.000000,72.000000,107.000000,4.000000,0.000000,1.000000,...,24.436850,12.973475,45.048650,21.116075,35.798500,47.000000,0.000000,15.870652,362.000000,1.880000
50%,10.000000,0.000000,65.000000,26.000000,68.000000,81.000000,114.000000,5.000000,0.000000,2.000000,...,28.794550,16.408600,56.518200,27.277150,44.819800,55.000000,1.000000,17.942687,476.000000,2.460000
75%,13.000000,1.000000,75.000000,30.000000,76.000000,90.500000,125.000000,6.000000,1.000000,2.000000,...,35.214650,21.983775,75.796050,37.777550,60.035800,64.000000,2.000000,21.579060,590.500000,3.090000
max,22.000000,1.000000,95.000000,50.000000,119.000000,138.000000,179.000000,28.000000,1.000000,3.000000,...,79.473800,52.527500,149.830000,97.923100,146.075000,100.000000,3.000000,59.132048,1200.000000,4.790000


In [76]:
# Mean

X_train_fimpute_mean = X_train.copy()

for column in X_train_fimpute_mean.columns:
    mean_val = np.mean(X_train[column])
    X_train_fimpute_mean.loc[X_train_fimpute_mean[column].isna(),column] = mean_val

X_train_fimpute_mean.describe()

<ipython-input-76-6a9d40dd8782>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.43358585858586' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train_fimpute_mean.loc[X_train_fimpute_mean[column].isna(),column] = mean_val
<ipython-input-76-6a9d40dd8782>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.37272727272727274' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train_fimpute_mean.loc[X_train_fimpute_mean[column].isna(),column] = mean_val


,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,...,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,10.433586,0.372727,65.069008,27.278508,69.242249,81.597236,116.591899,4.989233,0.476337,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,57.763622,1.060588,19.334696,469.910690,2.500797
std,3.574648,0.483591,9.213823,2.650007,10.873968,11.827929,13.998391,0.871935,0.379182,0.317126,...,6.887559,5.016538,16.640829,9.781378,13.839895,10.704254,0.999606,4.473526,81.478743,0.607654
min,5.000000,0.000000,25.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,1.000000,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,0.000000,0.000000,5.000000,0.580000
25%,8.000000,0.000000,61.000000,27.278508,64.000000,76.000000,109.000000,4.989233,0.000000,1.829567,...,28.801250,16.394825,56.796400,27.305300,44.819225,51.000000,0.000000,16.485531,469.910690,2.357500
50%,10.000000,0.000000,65.069008,27.278508,69.242249,81.597236,116.591899,4.989233,0.476337,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,57.763622,1.060588,19.334696,469.910690,2.500797
75%,13.000000,1.000000,68.000000,27.278508,72.000000,87.000000,121.000000,4.989233,1.000000,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,59.000000,2.000000,20.089901,469.910690,2.570000
max,22.000000,1.000000,95.000000,50.000000,119.000000,138.000000,179.000000,28.000000,1.000000,3.000000,...,79.473800,52.527500,149.830000,97.923100,146.075000,100.000000,3.000000,59.132048,1200.000000,4.790000


In [77]:
mean_scaler = StandardScaler()                  

X_train_fimpute_mean[X_train_fimpute_mean.columns] = mean_scaler.fit_transform(X_train_fimpute_mean[X_train_fimpute_mean.columns])
X_train_fimpute_mean.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,...,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03
mean,-1.928872e-16,-5.831474e-17,5.472615e-17,1.455177e-15,4.458835e-16,-1.247936e-15,6.573866e-16,-9.734077e-16,-8.679925e-17,-8.056406e-16,...,4.037175e-17,-8.182007e-16,-1.915415e-16,2.274275e-16,1.381611e-16,-6.064733e-16,-5.382900e-18,6.997769e-17,6.773482e-16,1.982701e-16
std,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,...,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00
min,-1.520226e+00,-7.708456e-01,-4.349342e+00,-3.501756e+00,-6.368512e+00,-4.616542e+00,-8.330002e+00,-5.722744e+00,-1.256379e+00,-2.616223e+00,...,-2.443542e+00,-2.675477e+00,-2.354323e+00,-2.717075e+00,-2.135461e+00,-1.846567e+00,-1.061140e+00,-4.322572e+00,-5.706634e+00,-3.161402e+00
25%,-6.808763e-01,-7.708456e-01,-4.416757e-01,1.340812e-15,-4.821525e-01,-4.732818e-01,-5.424093e-01,-1.018757e-15,-1.256379e+00,-7.002669e-16,...,-3.652939e-01,-3.311314e-01,-3.604029e-01,-4.012018e-01,-3.845026e-01,-6.319428e-01,-1.061140e+00,-6.369751e-01,6.977353e-16,-2.358501e-01
50%,-1.213100e-01,-7.708456e-01,0.000000e+00,1.340812e-15,0.000000e+00,-1.201618e-15,1.015306e-15,-1.018757e-15,0.000000e+00,-7.002669e-16,...,0.000000e+00,-7.082897e-16,-4.270415e-16,3.632578e-16,0.000000e+00,-6.638786e-16,0.000000e+00,0.000000e+00,6.977353e-16,0.000000e+00
75%,7.180394e-01,1.297277e+00,3.181483e-01,1.340812e-15,2.536424e-01,4.568379e-01,3.149403e-01,-1.018757e-15,1.381208e+00,-7.002669e-16,...,0.000000e+00,-7.082897e-16,-4.270415e-16,3.632578e-16,0.000000e+00,1.155180e-01,9.399011e-01,1.688380e-01,6.977353e-16,1.138994e-01
max,3.236088e+00,1.297277e+00,3.248898e+00,8.575207e+00,4.576437e+00,4.769211e+00,4.458797e+00,2.639378e+01,1.381208e+00,3.691220e+00,...,6.992748e+00,6.872489e+00,5.230986e+00,6.819326e+00,6.932646e+00,3.946255e+00,1.940421e+00,8.897317e+00,8.961620e+00,3.767753e+00


In [85]:
# Median

X_train_fimpute_median = X_train.copy()

for column in X_train_fimpute_median.columns:
    median_val = np.median(X_train.loc[X_train[column].notna()][column])
    X_train_fimpute_median.loc[X_train_fimpute_median[column].isna(),column] = median_val

X_train_fimpute_median.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,...,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,10.433586,0.372727,65.042172,26.289924,68.920707,81.447475,115.922980,4.997980,0.274495,1.954293,...,30.054459,17.233006,59.628686,29.247143,47.481258,56.818687,1.050505,19.012003,474.863636,2.482614
std,3.574648,0.483591,9.213885,2.703558,10.887576,11.830762,14.044273,0.871945,0.446316,0.325992,...,7.002103,5.083709,16.934060,9.979019,14.093285,10.784249,0.999861,4.511937,81.513283,0.607993
min,5.000000,0.000000,25.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,1.000000,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,0.000000,0.000000,5.000000,0.580000
25%,8.000000,0.000000,61.000000,26.000000,64.000000,76.000000,109.000000,5.000000,0.000000,2.000000,...,28.794550,16.394825,56.518200,27.277150,44.819225,51.000000,0.000000,16.485531,476.000000,2.357500
50%,10.000000,0.000000,65.000000,26.000000,68.000000,81.000000,114.000000,5.000000,0.000000,2.000000,...,28.794550,16.408600,56.518200,27.277150,44.819800,55.000000,1.000000,17.942687,476.000000,2.460000
75%,13.000000,1.000000,68.000000,26.000000,72.000000,87.000000,121.000000,5.000000,1.000000,2.000000,...,28.794550,16.417725,56.518200,27.277150,44.822725,59.000000,2.000000,20.089901,476.000000,2.570000
max,22.000000,1.000000,95.000000,50.000000,119.000000,138.000000,179.000000,28.000000,1.000000,3.000000,...,79.473800,52.527500,149.830000,97.923100,146.075000,100.000000,3.000000,59.132048,1200.000000,4.790000


In [86]:
median_scaler = StandardScaler()                  

X_train_fimpute_median[X_train_fimpute_median.columns] = median_scaler.fit_transform(X_train_fimpute_median[X_train_fimpute_median.columns])
X_train_fimpute_median.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,...,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03
mean,-1.928872e-16,-5.831474e-17,5.768674e-16,-9.895564e-16,3.445056e-16,3.750087e-16,4.077546e-16,2.435762e-16,-9.599504e-17,3.489913e-16,...,-4.108947e-16,6.100619e-17,3.732144e-16,-3.557199e-16,2.960595e-17,8.792069e-17,-9.330359e-17,4.611351e-16,-1.269467e-16,-4.665180e-16
std,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,...,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00
min,-1.520226e+00,-7.708456e-01,-4.346400e+00,-3.066689e+00,-6.331016e+00,-4.602776e+00,-8.255153e+00,-5.732710e+00,-6.151017e-01,-2.927717e+00,...,-2.223250e+00,-2.478267e+00,-2.126669e+00,-2.464623e+00,-1.908388e+00,-1.745236e+00,-1.050784e+00,-4.214244e+00,-5.764986e+00,-3.129731e+00
25%,-6.808763e-01,-7.708456e-01,-4.387598e-01,-1.072516e-01,-4.520133e-01,-4.605082e-01,-4.930020e-01,2.317183e-03,-6.151017e-01,1.402267e-01,...,-1.799557e-01,-1.648967e-01,-1.837054e-01,-1.974384e-01,-1.889105e-01,-5.396223e-01,-1.050784e+00,-5.600235e-01,1.394260e-02,-2.058074e-01
50%,-1.213100e-01,-7.708456e-01,-4.577553e-03,-1.072516e-01,-8.457561e-02,-3.782776e-02,-1.369400e-01,2.317183e-03,-6.151017e-01,1.402267e-01,...,-1.799557e-01,-1.621867e-01,-1.837054e-01,-1.974384e-01,-1.888697e-01,-1.686642e-01,-5.051845e-02,-2.370269e-01,1.394260e-02,-3.719862e-02
75%,7.180394e-01,1.297277e+00,3.210591e-01,-1.072516e-01,2.828620e-01,4.693887e-01,3.615468e-01,2.317183e-03,1.625747e+00,1.402267e-01,...,-1.799557e-01,-1.603916e-01,-1.837054e-01,-1.974384e-01,-1.886621e-01,2.022940e-01,9.497469e-01,2.389295e-01,1.394260e-02,1.437474e-01
max,3.236088e+00,1.297277e+00,3.251789e+00,8.771061e+00,4.600254e+00,4.780729e+00,4.491866e+00,2.638344e+01,1.625747e+00,3.208170e+00,...,7.058677e+00,6.943542e+00,5.327293e+00,6.882904e+00,6.996679e+00,4.004615e+00,1.950012e+00,8.893100e+00,8.897052e+00,3.795567e+00


In [33]:
#KNN
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=30, weights='distance') #30 has worked well in the past
X_train_fimpute_knn = pd.DataFrame(knn_imputer.fit_transform(X_train), columns = X_train.columns)
X_train_fimpute_knn.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,...,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,10.433586,0.372727,64.868609,26.892919,69.285364,81.418071,116.339899,4.873853,0.479768,1.818804,...,31.306746,18.019151,63.518488,31.116039,50.167154,58.083483,1.071311,19.468548,455.259767,2.515851
std,3.574648,0.483591,10.052045,4.016683,11.246181,12.216466,14.603324,0.977338,0.411427,0.351487,...,8.856623,6.465503,21.880104,12.498368,17.728622,11.898635,1.031655,4.773115,95.874235,0.667581
min,5.000000,0.000000,25.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,1.000000,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,0.000000,0.000000,5.000000,0.580000
25%,8.000000,0.000000,60.000000,24.000000,63.000000,74.000000,108.000000,4.584122,0.000000,1.725932,...,25.134004,13.122775,46.671500,21.763000,37.298568,50.000000,0.000000,16.278703,422.627536,2.130000
50%,10.000000,0.000000,65.000000,26.000000,68.380952,81.000000,114.291667,4.881910,0.478196,1.834561,...,28.786110,16.524600,57.392422,27.626150,45.390860,56.500000,1.000000,18.267153,454.300803,2.539943
75%,13.000000,1.000000,70.000000,28.961712,74.000000,88.000000,122.925000,5.146691,1.000000,2.000000,...,35.338329,21.844490,77.205750,37.821475,60.198483,62.925000,2.000000,21.717966,491.911210,2.778608
max,22.000000,1.000000,95.000000,50.000000,119.000000,138.000000,179.000000,28.000000,1.000000,3.000000,...,79.473800,52.527500,149.830000,97.923100,146.075000,100.000000,3.000000,59.132048,1200.000000,4.790000


In [45]:
knn_scaler = StandardScaler()                  

X_train_fimpute_knn[X_train_fimpute_knn.columns] = knn_scaler.fit_transform(X_train_fimpute_knn[X_train_fimpute_knn.columns])
X_train_fimpute_knn.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,...,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03
mean,-1.928872e-16,-5.831474e-17,3.624486e-16,5.436729e-16,-9.124015e-16,9.173358e-16,7.320743e-16,-8.455638e-16,-1.435440e-17,-3.382255e-16,...,-5.652044e-17,1.094523e-16,1.758414e-16,1.794300e-16,1.130409e-16,-3.009938e-16,2.870880e-17,1.901958e-16,3.570657e-16,-5.867360e-16
std,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,...,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00
min,-1.520226e+00,-7.708456e-01,-3.966719e+00,-2.214275e+00,-6.161568e+00,-4.455048e+00,-7.967679e+00,-4.987497e+00,-1.166254e+00,-2.329836e+00,...,-1.899129e+00,-2.070223e+00,-1.823731e+00,-2.117369e+00,-1.668584e+00,-1.688094e+00,-1.038570e+00,-4.079308e+00,-4.696951e+00,-2.900165e+00
25%,-6.808763e-01,-7.708456e-01,-4.844013e-01,-7.203169e-01,-5.589593e-01,-6.072957e-01,-5.711681e-01,-2.964867e-01,-1.166254e+00,-2.642593e-01,...,-6.970515e-01,-7.574035e-01,-7.700656e-01,-7.484354e-01,-7.259567e-01,-6.794480e-01,-1.038570e+00,-6.683785e-01,-3.404079e-01,-5.780568e-01
50%,-1.213100e-01,-7.708456e-01,1.307276e-02,-2.223307e-01,-8.042959e-02,-3.422625e-02,-1.402757e-01,8.244151e-03,-3.819653e-03,4.483639e-02,...,-2.846405e-01,-2.311869e-01,-2.800188e-01,-2.792629e-01,-2.694455e-01,-1.330979e-01,-6.913122e-02,-2.517323e-01,-1.000357e-02,3.609284e-02
75%,7.180394e-01,1.297277e+00,5.105468e-01,5.151151e-01,4.192740e-01,5.388432e-01,4.509886e-01,2.791990e-01,1.264619e+00,5.155775e-01,...,4.552629e-01,5.917285e-01,6.256365e-01,5.365727e-01,5.658983e-01,4.069482e-01,9.003074e-01,4.713276e-01,3.823350e-01,3.936456e-01
max,3.236088e+00,1.297277e+00,2.997917e+00,5.753504e+00,4.421138e+00,4.632197e+00,4.291353e+00,2.366538e+01,1.264619e+00,3.360991e+00,...,5.439222e+00,5.337977e+00,3.945247e+00,5.345938e+00,5.410458e+00,3.523245e+00,1.869746e+00,8.310821e+00,7.768868e+00,3.406980e+00


In [35]:
#MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iter_imputer = IterativeImputer(max_iter=10, random_state=42)
X_train_fimpute_mice = pd.DataFrame(iter_imputer.fit_transform(X_train), columns = X_train.columns)
X_train_fimpute_mice.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,...,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,10.433586,0.372727,64.818333,26.974426,69.403169,81.519388,116.805724,4.974030,0.480343,1.849340,...,31.365620,18.066786,63.749033,31.477562,50.138601,57.881433,1.073783,19.374281,468.721596,2.537644
std,3.574648,0.483591,9.805847,4.770198,11.011761,12.021297,14.228392,0.935016,0.389401,0.346190,...,7.099089,5.021442,18.068723,10.250418,13.839926,10.737030,1.022870,4.520383,89.856124,0.653839
min,5.000000,0.000000,-23.164848,4.967968,0.000000,27.000000,0.000000,0.000000,-0.172314,-1.772461,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,-0.278039,0.000000,5.000000,0.580000
25%,8.000000,0.000000,60.000000,24.000000,64.000000,74.477905,109.000000,4.865373,0.000000,1.766934,...,28.665877,16.394825,56.747700,27.190300,44.819225,51.000000,0.000000,16.485531,450.588533,2.150000
50%,10.000000,0.000000,64.765647,26.137767,69.000000,81.485263,115.604541,4.973003,0.485209,1.872826,...,31.347850,18.058831,63.734357,30.792761,50.135566,57.770114,1.000000,18.589876,469.207062,2.567600
75%,13.000000,1.000000,70.000000,29.035486,73.000000,87.000000,122.000000,5.081351,1.000000,2.000000,...,32.689980,18.232694,63.917531,33.007960,50.147145,60.000000,2.000000,21.015855,487.416029,2.886524
max,22.000000,1.000000,95.000000,56.944995,119.000000,138.000000,179.000000,28.000000,1.129295,3.000000,...,86.587576,52.527500,188.145195,111.835760,146.075000,100.000000,3.019947,59.132048,2154.275208,4.790000


Note that some imputed values were, e.g., <0, so clipping is required.

In [41]:
# Clip imputed values to original max and min
for column in X_train_fimpute_mice.columns:
    max_val = np.max(X_train[column])
    min_val = np.min(X_train[column])
    X_train_fimpute_mice.loc[X_train_fimpute_mice[column]>max_val,column] = max_val
    X_train_fimpute_mice.loc[X_train_fimpute_mice[column]<min_val, column] = min_val

X_train_fimpute_mice.describe()
    

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,...,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,10.433586,0.372727,64.830496,26.998653,69.403169,81.519388,116.805724,4.974030,0.480419,1.850041,...,31.363446,18.066786,63.672368,31.469455,50.138601,57.881433,1.073848,19.374281,468.480617,2.537644
std,3.574648,0.483591,9.726236,4.692416,11.011761,12.021297,14.228392,0.935016,0.389164,0.341634,...,7.083396,5.021442,17.644465,10.193244,13.839926,10.737030,1.022777,4.520383,86.553527,0.653839
min,5.000000,0.000000,25.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,1.000000,...,14.489000,4.635810,23.620100,4.655730,20.589200,38.000000,0.000000,0.000000,5.000000,0.580000
25%,8.000000,0.000000,60.000000,24.000000,64.000000,74.477905,109.000000,4.865373,0.000000,1.766934,...,28.665877,16.394825,56.747700,27.190300,44.819225,51.000000,0.000000,16.485531,450.588533,2.150000
50%,10.000000,0.000000,64.765647,26.137767,69.000000,81.485263,115.604541,4.973003,0.485209,1.872826,...,31.347850,18.058831,63.734357,30.792761,50.135566,57.770114,1.000000,18.589876,469.207062,2.567600
75%,13.000000,1.000000,70.000000,29.035486,73.000000,87.000000,122.000000,5.081351,1.000000,2.000000,...,32.689980,18.232694,63.917531,33.007960,50.147145,60.000000,2.000000,21.015855,487.416029,2.886524
max,22.000000,1.000000,95.000000,50.000000,119.000000,138.000000,179.000000,28.000000,1.000000,3.000000,...,79.473800,52.527500,149.830000,97.923100,146.075000,100.000000,3.000000,59.132048,1200.000000,4.790000


In [46]:
mice_scaler = StandardScaler()                  

X_train_fimpute_mice[X_train_fimpute_mice.columns] = mice_scaler.fit_transform(X_train_fimpute_mice[X_train_fimpute_mice.columns])
X_train_fimpute_mice.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,...,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03,3.960000e+03
mean,-1.928872e-16,-5.831474e-17,8.343494e-16,5.921189e-17,2.866394e-16,4.893953e-16,-3.545985e-16,6.948426e-16,-1.282924e-16,1.058637e-16,...,-3.184882e-16,-1.004808e-16,1.040694e-16,-4.135861e-16,1.964758e-16,-1.821214e-16,-5.741759e-17,9.796877e-16,4.831152e-16,-1.578984e-16
std,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,...,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00,1.000126e+00
min,-1.520226e+00,-7.708456e-01,-4.095678e+00,-1.917944e+00,-6.303437e+00,-4.535806e+00,-8.210378e+00,-5.320400e+00,-1.234646e+00,-2.488480e+00,...,-2.382555e+00,-2.675063e+00,-2.270249e+00,-2.630871e+00,-2.135353e+00,-1.851904e+00,-1.050066e+00,-4.286523e+00,-5.355520e+00,-2.994455e+00
25%,-6.808763e-01,-7.708456e-01,-4.967087e-01,-6.391231e-01,-4.907346e-01,-5.858247e-01,-5.486713e-01,-1.162227e-01,-1.234646e+00,-2.432958e-01,...,-3.808780e-01,-3.330064e-01,-3.925051e-01,-4.198561e-01,-3.843986e-01,-6.409875e-01,-1.050066e+00,-6.391307e-01,-2.067431e-01,-5.929487e-01
50%,-1.213100e-01,-7.708456e-01,-6.668295e-03,-1.834866e-01,-3.661725e-02,-2.839099e-03,-8.443218e-02,-1.098129e-03,1.230976e-02,6.670247e-02,...,-2.202046e-03,-1.584385e-03,3.513687e-03,-6.639494e-02,-2.193295e-04,-1.036914e-02,-7.221268e-02,-1.735482e-01,8.394070e-03,4.582149e-02
75%,7.180394e-01,1.297277e+00,5.315681e-01,4.341239e-01,3.266766e-01,4.559661e-01,3.651103e-01,1.147947e-01,1.335289e+00,4.390017e-01,...,1.872974e-01,3.304415e-02,1.389639e-02,1.509529e-01,6.173593e-04,1.973390e-01,9.056408e-01,3.631950e-01,2.187987e-01,5.336544e-01
max,3.236088e+00,1.297277e+00,3.102260e+00,4.902432e+00,4.504556e+00,4.698973e+00,4.371691e+00,2.462940e+01,1.335289e+00,3.366483e+00,...,6.792848e+00,6.863580e+00,4.883600e+00,6.520205e+00,6.932733e+00,3.923235e+00,1.883494e+00,8.796331e+00,8.452709e+00,3.445252e+00


In [44]:
#RF - not used, cannot import fancyimpute
#from fancyimpute import RandomForestImputer

#rf_imputer = RandomForestImputer()
#X_train_fimpute_rf = pd.DataFrame(rf_imputer.fit_transform(X_train), columns = X_train.columns)
#X_train_fimpute_rf.describe()

In [ ]:
# Autoencoder - tricky
# How to handle passing missing values, will it regenerate missing values?
# https://www.xyonix.com/blog/filling-in-the-gaps-ai-powered-data-imputation-using-autoencoders

#stacked_encoder = keras.models.Sequential([
#    keras.layers.Flatten(input_shape=[28, 28]),
#    keras.layers.Dense(100, activation="selu"),
#    keras.layers.Dense(30, activation="selu"),
#])

#stacked_decoder = keras.models.Sequential([
#    keras.layers.Dense(100, activation="selu", input_shape=[30]),
#    keras.layers.Dense(28 * 28, activation="sigmoid"),
#    keras.layers.Reshape([28, 28])
#])

#stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

# Repeat imputation and scaling for test data

In [48]:
X_test.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,8.000000,5.000000,10.000000,12.000000,11.000000,3.0,13.000000,5.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,10.00000,16.000000,13.000000,3.000000,10.000000
mean,10.750000,0.400000,62.500000,25.400000,65.300000,81.666667,117.545455,5.0,0.461538,1.600000,...,28.486750,15.457795,56.989275,25.985962,45.167825,52.30000,1.437500,19.835939,454.000000,2.239100
std,3.725799,0.502625,11.275764,3.130495,7.528465,9.316001,21.262002,1.0,0.518875,0.547723,...,5.099449,4.021153,14.490362,7.479799,11.940000,7.02456,1.152895,4.927625,122.503061,1.102044
min,5.000000,0.000000,50.000000,22.000000,57.000000,70.000000,95.000000,4.0,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.00000,0.000000,14.035590,332.000000,1.040000
25%,9.000000,0.000000,51.000000,24.000000,60.250000,74.500000,102.500000,4.5,0.000000,1.000000,...,24.230725,13.815400,44.627250,19.801775,37.245575,47.75000,0.000000,16.861286,392.500000,1.232500
50%,10.000000,0.000000,63.000000,24.000000,62.000000,80.000000,116.000000,5.0,0.000000,2.000000,...,29.470400,16.402450,59.199050,26.337750,46.608850,53.50000,2.000000,18.292347,453.000000,2.255000
75%,12.250000,1.000000,71.000000,27.000000,69.500000,90.250000,119.500000,5.5,1.000000,2.000000,...,31.398725,17.674625,65.222050,30.421100,51.860475,55.75000,2.000000,21.079065,515.000000,2.877750
max,19.000000,1.000000,80.000000,30.000000,80.000000,97.000000,163.000000,6.0,1.000000,2.000000,...,36.057200,20.902000,79.698200,36.223200,63.126500,64.00000,3.000000,30.094649,577.000000,4.110000


In [87]:
# Mean

X_test_fimpute_mean = X_test.copy()

for column in X_test_fimpute_mean.columns:
    mean_val = np.mean(X_train[column])
    X_test_fimpute_mean.loc[X_test_fimpute_mean[column].isna(),column] = mean_val

X_test_fimpute_mean.describe()

<ipython-input-87-00c18b4eaf9e>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.43358585858586' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_test_fimpute_mean.loc[X_test_fimpute_mean[column].isna(),column] = mean_val
<ipython-input-87-00c18b4eaf9e>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.37272727272727274' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_test_fimpute_mean.loc[X_test_fimpute_mean[column].isna(),column] = mean_val


,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.750000,0.400000,64.041405,26.808881,67.271124,81.638895,117.116355,4.990848,0.466718,1.772175,...,30.184849,17.016567,60.471538,29.131852,48.151147,55.031811,1.362118,19.660504,467.524087,2.369949
std,3.725799,0.502625,6.964871,1.661213,5.562121,7.088500,15.432754,0.324467,0.412423,0.271218,...,3.406479,2.768095,9.266454,5.249495,7.666107,5.588314,1.035987,3.923753,40.170382,0.770268
min,5.000000,0.000000,50.000000,22.000000,57.000000,70.000000,95.000000,4.000000,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.000000,0.000000,14.035590,332.000000,1.040000
25%,9.000000,0.000000,65.069008,27.278508,62.500000,78.250000,114.250000,4.989233,0.000000,1.829567,...,30.771225,17.301750,61.960860,28.195950,49.410297,53.750000,0.750000,17.182707,469.910690,2.297500
50%,10.000000,0.000000,65.069008,27.278508,69.242249,81.597236,116.591899,4.989233,0.476337,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,57.763622,1.060588,19.334696,469.910690,2.500797
75%,12.250000,1.000000,65.069008,27.278508,69.242249,81.947927,116.693924,4.989233,1.000000,1.829567,...,31.316916,18.055748,62.793046,31.229111,50.140029,57.763622,2.000000,19.718860,469.910690,2.500797
max,19.000000,1.000000,80.000000,30.000000,80.000000,97.000000,163.000000,6.000000,1.000000,2.000000,...,36.057200,20.902000,79.698200,36.223200,63.126500,64.000000,3.000000,30.094649,577.000000,4.110000


In [88]:
X_test_fimpute_mean[X_test_fimpute_mean.columns] = mean_scaler.transform(X_test_fimpute_mean[X_test_fimpute_mean.columns])
X_test_fimpute_mean.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,2.000000e+01,20.000000,2.000000e+01,2.000000e+01,2.000000e+01,20.000000,2.000000e+01,...,20.000000,2.000000e+01,2.000000e+01,2.000000e+01,20.000000,2.000000e+01,20.000000,20.000000,2.000000e+01,20.000000
mean,0.088527,0.056403,-0.111543,-1.772396e-01,-0.181293,3.522468e-03,3.747014e-02,1.852520e-03,-0.025370,-1.809975e-01,...,-0.164385,-2.071773e-01,-1.395244e-01,-2.144406e-01,-0.143725,-2.552402e-01,0.301687,0.072840,-2.929482e-02,-0.215361
std,1.042416,1.039489,0.756011,6.269504e-01,0.511573,5.993776e-01,1.102605e+00,3.721696e-01,1.087802,8.553472e-01,...,0.494647,5.518635e-01,5.569208e-01,5.367503e-01,0.553984,5.221307e-01,1.036526,0.877216,4.930790e-01,1.267768
min,-1.520226,-0.770846,-1.635685,-1.992136e+00,-1.125973,-9.806198e-01,-1.542651e+00,-1.134669e+00,-1.256379,-2.616223e+00,...,-1.492984,-1.826268e+00,-1.434926e+00,-1.617401e+00,-1.668202,-1.659701e+00,-1.061140,-1.184697,-1.692811e+00,-2.404297
25%,-0.401093,-0.770846,0.000000,1.340812e-15,-0.620114,-2.830300e-01,-1.673189e-01,-1.018757e-15,-1.256379,-7.002669e-16,...,-0.079238,-1.503215e-01,-5.001503e-02,-3.101346e-01,-0.052733,-3.750032e-01,-0.310749,-0.481111,6.977353e-16,-0.334603
50%,-0.121310,-0.770846,0.000000,1.340812e-15,0.000000,-1.201618e-15,1.015306e-15,-1.018757e-15,0.000000,-7.002669e-16,...,0.000000,-7.082897e-16,-4.270415e-16,3.632578e-16,0.000000,-6.638786e-16,0.000000,0.000000,6.977353e-16,0.000000
75%,0.508202,1.297277,0.000000,1.340812e-15,0.000000,2.965314e-02,7.289271e-03,-1.018757e-15,1.381208,-7.002669e-16,...,0.000000,-7.082897e-16,-4.270415e-16,3.632578e-16,0.000000,-6.638786e-16,0.939901,0.085886,6.977353e-16,0.000000
max,2.396738,1.297277,1.620704,1.027105e+00,0.989437,1.302401e+00,3.315664e+00,1.159369e+00,1.381208,5.374987e-01,...,0.688326,5.674453e-01,1.016012e+00,5.106356e-01,0.938454,5.826810e-01,1.940421,2.405555,1.314488e+00,2.648554


In [89]:
# Median

X_test_fimpute_median = X_test.copy()

for column in X_test_fimpute_median.columns:
    median_val = np.median(X_train.loc[X_train[column].notna()][column])
    X_test_fimpute_median.loc[X_test_fimpute_median[column].isna(),column] = median_val

X_test_fimpute_median.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000
mean,10.750000,0.400000,64.000000,25.850000,66.650000,81.40000,115.950000,5.000000,0.300000,1.900000,...,28.671430,16.028278,56.706630,26.760675,44.959010,53.650000,1.35000,19.173301,472.700000,2.349550
std,3.725799,0.502625,6.958524,1.460894,5.363375,7.09633,15.530869,0.324443,0.470162,0.307794,...,3.099112,2.487093,8.798503,4.586216,7.249416,5.029126,1.03999,4.024182,40.554187,0.766897
min,5.000000,0.000000,50.000000,22.000000,57.000000,70.00000,95.000000,4.000000,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.000000,0.00000,14.035590,332.000000,1.040000
25%,9.000000,0.000000,65.000000,26.000000,62.500000,78.25000,112.750000,5.000000,0.000000,2.000000,...,28.794550,16.408600,56.518200,27.077813,44.819800,53.750000,0.75000,17.182707,476.000000,2.297500
50%,10.000000,0.000000,65.000000,26.000000,68.000000,81.00000,114.000000,5.000000,0.000000,2.000000,...,28.794550,16.408600,56.518200,27.277150,44.819800,55.000000,1.00000,17.942687,476.000000,2.460000
75%,12.250000,1.000000,65.000000,26.000000,68.000000,81.50000,116.250000,5.000000,1.000000,2.000000,...,28.794550,16.408600,57.122100,27.277150,45.114000,55.000000,2.00000,19.718860,476.000000,2.460000
max,19.000000,1.000000,80.000000,30.000000,80.000000,97.00000,163.000000,6.000000,1.000000,2.000000,...,36.057200,20.902000,79.698200,36.223200,63.126500,64.000000,3.00000,30.094649,577.000000,4.110000


In [90]:
X_test_fimpute_median[X_test_fimpute_median.columns] = median_scaler.transform(X_test_fimpute_median[X_test_fimpute_median.columns])
X_test_fimpute_median.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.088527,0.056403,-0.113123,-0.162741,-0.208586,-0.004013,0.001924,0.002317,0.057153,-0.166568,...,-0.197541,-0.237008,-0.172577,-0.249201,-0.178991,-0.293863,0.299574,0.035754,-0.026547,-0.218885
std,1.042416,1.039489,0.755317,0.540428,0.492676,0.599896,1.105990,0.372138,1.053563,0.944293,...,0.442653,0.489290,0.519640,0.459644,0.514453,0.466399,1.040266,0.892009,0.497579,1.261518
min,-1.520226,-0.770846,-1.632761,-1.586970,-1.095029,-0.967725,-1.489975,-1.144688,-0.615102,-2.927717,...,-1.288241,-1.640278,-1.223192,-1.386729,-1.449530,-1.559757,-1.050784,-1.103083,-1.752864,-2.373047
25%,-0.401093,-0.770846,-0.004578,-0.107252,-0.589802,-0.270302,-0.225955,0.002317,-0.615102,0.140227,...,-0.179956,-0.162187,-0.183705,-0.217417,-0.188870,-0.284589,-0.300585,-0.405486,0.013943,-0.304505
50%,-0.121310,-0.770846,-0.004578,-0.107252,-0.084576,-0.037828,-0.136940,0.002317,-0.615102,0.140227,...,-0.179956,-0.162187,-0.183705,-0.197438,-0.188870,-0.168664,-0.050518,-0.237027,0.013943,-0.037199
75%,0.508202,1.297277,-0.004578,-0.107252,-0.084576,0.004440,0.023288,0.002317,1.625747,0.140227,...,-0.179956,-0.162187,-0.148039,-0.197438,-0.167992,-0.168664,0.949747,0.156683,0.013943,-0.037199
max,2.396738,1.297277,1.623606,1.372467,1.017737,1.314750,3.352467,1.149323,1.625747,0.140227,...,0.857385,0.721807,1.185306,0.699161,1.110261,0.665992,1.950012,2.456604,1.253161,2.676992


In [47]:
# KNN

X_test_fimpute_knn = pd.DataFrame(knn_imputer.transform(X_test), columns = X_test.columns)
X_test_fimpute_knn.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.750000,0.400000,63.893374,27.232890,68.844066,80.936993,117.704557,4.774470,0.458276,1.774232,...,31.231725,18.052421,64.926761,31.004235,51.039191,56.536945,1.486818,20.450175,444.561572,2.348645
std,3.725799,0.502625,7.516838,3.226048,7.968698,7.446801,15.731184,0.520071,0.444699,0.292443,...,6.166867,4.895591,16.749135,9.795177,13.123497,8.495075,1.094726,4.340366,62.522798,0.777565
min,5.000000,0.000000,50.000000,22.000000,57.000000,70.000000,95.000000,3.626199,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.000000,0.000000,14.035590,321.347363,1.040000
25%,9.000000,0.000000,60.750000,24.555913,62.500000,76.083333,111.040392,4.583752,0.000000,1.722296,...,28.240784,15.782166,58.328499,26.278089,45.649286,52.999951,0.575000,17.182707,425.606651,2.135671
50%,10.000000,0.000000,65.716667,27.129780,69.783333,79.283333,116.500000,4.783606,0.408974,1.830257,...,30.648850,17.878862,63.409230,29.494703,49.648857,55.752882,2.000000,19.924733,448.875060,2.447650
75%,12.250000,1.000000,68.087937,29.515545,72.923577,84.038228,120.425000,5.019058,1.000000,2.000000,...,34.332187,20.204950,71.102672,35.591100,56.705428,60.325000,2.083333,22.607081,483.631601,2.598476
max,19.000000,1.000000,80.000000,34.000000,90.421565,97.000000,163.000000,6.000000,1.000000,2.033333,...,49.499940,29.466295,104.393153,55.815855,83.330445,75.000000,3.000000,30.094649,577.000000,4.110000


In [49]:
X_test_fimpute_knn[X_test_fimpute_knn.columns] = knn_scaler.transform(X_test_fimpute_knn[X_test_fimpute_knn.columns])
X_test_fimpute_knn.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.088527,0.056403,-0.097031,0.084650,-0.039245,-0.039384,0.093460,-0.101700,-0.052244,-0.126827,...,-0.008472,0.005146,0.064371,-0.008947,0.049194,-0.129993,0.402809,0.205683,-0.111600,-0.250498
std,1.042416,1.039489,0.747886,0.803264,0.708659,0.609648,1.077369,0.532198,1.081008,0.832121,...,0.696388,0.757282,0.765593,0.783815,0.740337,0.714044,1.061269,0.909451,0.652216,1.164897
min,-1.520226,-0.770846,-1.479349,-1.218303,-1.092541,-0.934764,-1.461489,-1.276745,-1.166254,-2.329836,...,-1.159905,-1.411328,-1.124487,-1.256750,-1.303817,-1.519987,-1.038570,-1.138385,-1.396927,-2.211023
25%,-0.401093,-0.770846,-0.409780,-0.581898,-0.603424,-0.436739,-0.362943,-0.296866,-1.166254,-0.274606,...,-0.346221,-0.346031,-0.237231,-0.387135,-0.254867,-0.427291,-0.481143,-0.478960,-0.309331,-0.569561
50%,-0.121310,-0.770846,0.084377,0.058977,0.044285,-0.174765,0.010965,-0.092352,-0.172089,0.032588,...,-0.074292,-0.021701,-0.004994,-0.129740,-0.029239,-0.195896,0.900307,0.095586,-0.066603,-0.102175
75%,0.508202,1.297277,0.320306,0.653016,0.323547,0.214505,0.279773,0.148590,1.264619,0.515578,...,0.341645,0.338114,0.346668,0.358097,0.368844,0.188408,0.981094,0.657627,0.295965,0.123782
max,2.396738,1.297277,1.505495,1.769614,1.879649,1.275647,3.195573,1.152405,1.264619,0.610425,...,2.054450,1.770719,1.868356,1.976493,1.870844,1.421899,1.869746,2.226521,1.269951,2.388249


In [51]:
# MICE

X_test_fimpute_mice = pd.DataFrame(iter_imputer.transform(X_test), columns = X_test.columns)
X_test_fimpute_mice.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.750000,0.400000,64.384331,27.189833,68.802755,81.222964,118.182086,4.880596,0.444288,1.781648,...,29.934581,17.000002,61.027601,29.569552,48.151484,55.211312,1.480745,20.096235,456.502406,2.327285
std,3.725799,0.502625,7.793448,4.183181,8.019925,7.577237,15.582534,0.569945,0.414281,0.303869,...,3.557039,2.763017,9.424007,5.537355,7.666200,5.714718,1.052810,4.002297,46.087711,0.794676
min,5.000000,0.000000,50.000000,15.998384,57.000000,70.000000,95.000000,3.120483,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.000000,0.000000,14.035590,332.000000,1.040000
25%,9.000000,0.000000,60.956539,24.000000,62.500000,75.750000,112.206733,4.864217,0.000000,1.757357,...,28.861600,17.301750,62.397428,28.195950,49.404827,53.750000,0.750000,17.182707,451.607831,2.018671
50%,10.000000,0.000000,65.350489,27.875495,69.550418,80.357634,117.000000,4.964190,0.392398,1.821179,...,30.141796,18.005161,63.692298,30.880738,50.138271,57.487605,2.000000,19.641881,463.273518,2.375526
75%,12.250000,1.000000,69.138030,29.819804,71.139010,85.722666,121.927123,5.076138,1.000000,1.956164,...,32.380737,18.105431,63.812779,33.146024,50.148196,58.349053,2.029086,21.354441,473.176378,2.666661
max,19.000000,1.000000,80.000000,35.572032,92.524660,97.000000,163.000000,6.000000,1.000000,2.192637,...,36.057200,20.902000,79.698200,36.223200,63.126500,64.000000,3.000000,30.094649,577.000000,4.110000


In [52]:
# Clip imputed values to original max and min
for column in X_test_fimpute_mice.columns:
    max_val = np.max(X_train[column])
    min_val = np.min(X_train[column])
    X_test_fimpute_mice.loc[X_test_fimpute_mice[column]>max_val,column] = max_val
    X_test_fimpute_mice.loc[X_test_fimpute_mice[column]<min_val, column] = min_val

X_test_fimpute_mice.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.750000,0.400000,64.384331,27.289913,68.802755,81.222964,118.182086,4.880596,0.444288,1.781648,...,29.934581,17.000002,61.027601,29.569552,48.151484,55.211312,1.480745,20.096235,456.502406,2.327285
std,3.725799,0.502625,7.793448,3.916801,8.019925,7.577237,15.582534,0.569945,0.414281,0.303869,...,3.557039,2.763017,9.424007,5.537355,7.666200,5.714718,1.052810,4.002297,46.087711,0.794676
min,5.000000,0.000000,50.000000,18.000000,57.000000,70.000000,95.000000,3.120483,0.000000,1.000000,...,21.035200,8.895360,38.917700,15.410700,27.055200,40.000000,0.000000,14.035590,332.000000,1.040000
25%,9.000000,0.000000,60.956539,24.000000,62.500000,75.750000,112.206733,4.864217,0.000000,1.757357,...,28.861600,17.301750,62.397428,28.195950,49.404827,53.750000,0.750000,17.182707,451.607831,2.018671
50%,10.000000,0.000000,65.350489,27.875495,69.550418,80.357634,117.000000,4.964190,0.392398,1.821179,...,30.141796,18.005161,63.692298,30.880738,50.138271,57.487605,2.000000,19.641881,463.273518,2.375526
75%,12.250000,1.000000,69.138030,29.819804,71.139010,85.722666,121.927123,5.076138,1.000000,1.956164,...,32.380737,18.105431,63.812779,33.146024,50.148196,58.349053,2.029086,21.354441,473.176378,2.666661
max,19.000000,1.000000,80.000000,35.572032,92.524660,97.000000,163.000000,6.000000,1.000000,2.192637,...,36.057200,20.902000,79.698200,36.223200,63.126500,64.000000,3.000000,30.094649,577.000000,4.110000


In [53]:
X_test_fimpute_mice[X_test_fimpute_mice.columns] = mice_scaler.transform(X_test_fimpute_mice[X_test_fimpute_mice.columns])
X_test_fimpute_mice.describe()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,FGC-FGC_CU_Zone,FGC-FGC_GSND_Zone,...,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-BMI_Calc,Fitness_Endurance-Time_Sec_Calc,PAQ_Total
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.088527,0.056403,-0.045878,0.062078,-0.054532,-0.024661,0.096746,-0.099940,-0.092855,-0.200220,...,-0.201746,-0.212473,-0.149911,-0.186412,-0.143597,-0.248715,0.397885,0.159731,-0.138408,-0.321770
std,1.042416,1.039489,0.801382,0.834814,0.728397,0.630397,1.095310,0.609633,1.064675,0.889571,...,0.502229,0.550313,0.534173,0.543306,0.553989,0.532311,1.029493,0.885501,0.532544,1.215554
min,-1.520226,-0.770846,-1.524985,-1.917944,-1.126499,-0.958369,-1.532744,-1.982619,-1.234646,-2.488480,...,-1.458277,-1.826683,-1.403148,-1.575630,-1.668095,-1.665609,-1.050066,-1.181175,-1.577034,-2.290829
25%,-0.401093,-0.770846,-0.398350,-0.639123,-0.626970,-0.479991,-0.323267,-0.117459,-1.234646,-0.271331,...,-0.353243,-0.152373,-0.072266,-0.321185,-0.053025,-0.384832,-0.316676,-0.484882,-0.194965,-0.793832
50%,-0.121310,-0.770846,0.053470,0.186887,0.013374,-0.096654,0.013656,-0.010525,-0.226210,-0.084494,...,-0.172488,-0.012274,0.001130,-0.057763,-0.000024,-0.036684,0.905641,0.059206,-0.060168,-0.247980
75%,0.508202,1.297277,0.442934,0.601291,0.157655,0.349697,0.359988,0.109218,1.335289,0.310673,...,0.143634,0.007697,0.007959,0.164499,0.000693,0.043558,0.934083,0.438107,0.054260,0.197347
max,2.396738,1.297277,1.559845,1.827302,2.099974,1.287928,3.247037,1.097415,1.335289,1.002942,...,0.662725,0.564693,0.908379,0.466421,0.938555,0.569929,1.883494,2.371861,1.253942,2.405110


# Use labelled data only

For each imputation approach, we have:
* X_train, X_test
* Common y_train

1. We extract labelled data from each X_train and common y_train
2. Fit model on X_train_labelled, common y_train_labelled
3. Predict on X_test

In [105]:
y_test = test_data[['id']]

In [106]:
# Constant model which we will use for testing all imputation approaches
xgb_impute_model = XGBRegressor(random_state=42, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=100, subsample=0.7)

In [107]:
#Mean
X_train_labelled = X_train_fimpute_mean.loc[y_train.notna()]
y_train_labelled = y_train[y_train.notna()]
print("Size of labelled train data set is: ", (X_train_labelled.shape, y_train_labelled.shape))

Size of labelled train data set is:  ((2736, 34), (2736,))


In [108]:
xgb_impute_model.fit(X_train_labelled, y_train_labelled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [109]:
y_test['PCIAT-PCIAT_Total_mean'] = xgb_impute_model.predict(X_test_fimpute_mean)
y_test.head(10)

<ipython-input-109-f53104586a7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PCIAT-PCIAT_Total_mean'] = xgb_impute_model.predict(X_test_fimpute_mean)


,id,PCIAT-PCIAT_Total_mean
0,00008ff9,28.097628
1,000fd460,13.788895
2,00105258,30.657166
3,00115b9f,20.609028
4,0016bb22,35.410194
5,001f3379,29.594351
6,0038ba98,34.180454
7,0068a485,28.730440
8,0069fbed,41.061615
9,0083e397,34.462418


In [110]:
#Median
X_train_labelled = X_train_fimpute_median.loc[y_train.notna()]
y_train_labelled = y_train[y_train.notna()]
print("Size of labelled train data set is: ", (X_train_labelled.shape, y_train_labelled.shape))

Size of labelled train data set is:  ((2736, 34), (2736,))


In [111]:
xgb_impute_model.fit(X_train_labelled, y_train_labelled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [112]:
y_test['PCIAT-PCIAT_Total_median'] = xgb_impute_model.predict(X_test_fimpute_median)
y_test.head(10)

<ipython-input-112-ce9fa5b6e266>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PCIAT-PCIAT_Total_median'] = xgb_impute_model.predict(X_test_fimpute_median)


,id,PCIAT-PCIAT_Total_mean,PCIAT-PCIAT_Total_median
0,00008ff9,28.097628,28.710884
1,000fd460,13.788895,16.450972
2,00105258,30.657166,30.761959
3,00115b9f,20.609028,19.592257
4,0016bb22,35.410194,34.400955
5,001f3379,29.594351,31.051321
6,0038ba98,34.180454,33.795845
7,0068a485,28.730440,28.209627
8,0069fbed,41.061615,39.437405
9,0083e397,34.462418,33.939198


In [113]:
#KNN
X_train_labelled = X_train_fimpute_knn.loc[y_train.notna()]
y_train_labelled = y_train[y_train.notna()]
print("Size of labelled train data set is: ", (X_train_labelled.shape, y_train_labelled.shape))

Size of labelled train data set is:  ((2736, 34), (2736,))


In [114]:
xgb_impute_model.fit(X_train_labelled, y_train_labelled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [115]:
y_test['PCIAT-PCIAT_Total_knn'] = xgb_impute_model.predict(X_test_fimpute_knn)
y_test.head(10)

<ipython-input-115-c8f8f6503990>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PCIAT-PCIAT_Total_knn'] = xgb_impute_model.predict(X_test_fimpute_knn)


,id,PCIAT-PCIAT_Total_mean,PCIAT-PCIAT_Total_median,PCIAT-PCIAT_Total_knn
0,00008ff9,28.097628,28.710884,27.825579
1,000fd460,13.788895,16.450972,14.881695
2,00105258,30.657166,30.761959,30.428879
3,00115b9f,20.609028,19.592257,21.287128
4,0016bb22,35.410194,34.400955,43.761642
5,001f3379,29.594351,31.051321,29.157087
6,0038ba98,34.180454,33.795845,33.366413
7,0068a485,28.730440,28.209627,32.692314
8,0069fbed,41.061615,39.437405,43.493179
9,0083e397,34.462418,33.939198,40.677105


In [116]:
#MICE
X_train_labelled = X_train_fimpute_mice.loc[y_train.notna()]
y_train_labelled = y_train[y_train.notna()]
print("Size of labelled train data set is: ", (X_train_labelled.shape, y_train_labelled.shape))

Size of labelled train data set is:  ((2736, 34), (2736,))


In [117]:
xgb_impute_model.fit(X_train_labelled, y_train_labelled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [118]:
y_test['PCIAT-PCIAT_Total_mice'] = xgb_impute_model.predict(X_test_fimpute_mice)
y_test.head(10)

<ipython-input-118-ec1a1babdc52>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PCIAT-PCIAT_Total_mice'] = xgb_impute_model.predict(X_test_fimpute_mice)


,id,PCIAT-PCIAT_Total_mean,PCIAT-PCIAT_Total_median,PCIAT-PCIAT_Total_knn,PCIAT-PCIAT_Total_mice
0,00008ff9,28.097628,28.710884,27.825579,26.997581
1,000fd460,13.788895,16.450972,14.881695,16.266987
2,00105258,30.657166,30.761959,30.428879,29.768028
3,00115b9f,20.609028,19.592257,21.287128,19.785748
4,0016bb22,35.410194,34.400955,43.761642,36.034180
5,001f3379,29.594351,31.051321,29.157087,30.798737
6,0038ba98,34.180454,33.795845,33.366413,32.711647
7,0068a485,28.730440,28.209627,32.692314,30.812389
8,0069fbed,41.061615,39.437405,43.493179,42.307667
9,0083e397,34.462418,33.939198,40.677105,36.498135


# Testing and choosing the imputation approach

In [119]:
y_test['PCIAT-PCIAT_Total'] = y_test.apply(
    #lambda row: (row['PCIAT-PCIAT_Total_mean']),
    #lambda row: (row['PCIAT-PCIAT_Total_median']),
    #lambda row: (row['PCIAT-PCIAT_Total_knn']),
    lambda row: (row['PCIAT-PCIAT_Total_mice']),
    
axis=1)

y_test['sii'] = y_test.apply(lambda row: 0 if row['PCIAT-PCIAT_Total']<=30 else 
                             (1 if row['PCIAT-PCIAT_Total']<50 else (
                                2 if row['PCIAT-PCIAT_Total']<80 else (3)
                            )), axis=1)

y_test.head()

<ipython-input-119-100a86424ffd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PCIAT-PCIAT_Total'] = y_test.apply(
<ipython-input-119-100a86424ffd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['sii'] = y_test.apply(lambda row: 0 if row['PCIAT-PCIAT_Total']<=30 else


,id,PCIAT-PCIAT_Total_mean,PCIAT-PCIAT_Total_median,PCIAT-PCIAT_Total_knn,PCIAT-PCIAT_Total_mice,PCIAT-PCIAT_Total,sii
0,00008ff9,28.097628,28.710884,27.825579,26.997581,28.097628,0
1,000fd460,13.788895,16.450972,14.881695,16.266987,13.788895,0
2,00105258,30.657166,30.761959,30.428879,29.768028,30.657166,1
3,00115b9f,20.609028,19.592257,21.287128,19.785748,20.609028,0
4,0016bb22,35.410194,34.400955,43.761642,36.034180,35.410194,1


In [120]:
solution = y_test[['id','sii']]
solution.to_csv("submission.csv", index=False)

Scoring (QWK on private and public datasets):
* Mean: 0.355, 0.393
* Median: 0.351, 0.390
* KNN: 0.373, 0.382
* MICE: 0.388, 0.377